[View in Colaboratory](https://colab.research.google.com/github/neerajtiwari360/Deep_Learning./blob/Object_Classifier_N_Layer_NN/LeNet_5_MNIST.ipynb)

# **LeNet-5 CNN on MNIST dataset**


In [0]:
from tensorflow.examples.tutorials.mnist import input_data
import tensorflow as tf
import random
import numpy as np
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from tensorflow.contrib.layers import flatten

%matplotlib inline


**Read the dataset **

In [0]:
mnist = input_data.read_data_sets("MNIST_data/", reshape=False)

**Check the size of training/validation/testing dataset**

In [0]:
X_train, y_train           = mnist.train.images, mnist.train.labels
X_validation, y_validation = mnist.validation.images, mnist.validation.labels
X_test, y_test             = mnist.test.images, mnist.test.labels

print()
print("Image Shape: " + str(X_train[0].shape))
print()
print("Training Set: " + str(len(X_train))+ " samples")
print("Validation Set: "+ str(len(X_validation))+" samples")
print("Test Set: "+ str(len(X_test))+ " samples")

**Padding the dataset to avoid the loss of data at the edges**

In [0]:
# Pad images with 0s
X_train      = np.pad(X_train, ((0,0),(2,2),(2,2),(0,0)), 'constant')
X_validation = np.pad(X_validation, ((0,0),(2,2),(2,2),(0,0)), 'constant')
X_test       = np.pad(X_test, ((0,0),(2,2),(2,2),(0,0)), 'constant')
    
print("Updated Image Shape: "+ str(X_train[0].shape))

**Check any one training data image & label**

In [0]:
index = 100
image = X_train[index].squeeze()
plt.imshow(image, cmap="gray")
print(y_train[index])

**Shuffle the traning data to avoid overfitting**

In [0]:
X_train, y_train = shuffle(X_train, y_train)

**Forward Helper function**
>Implements the forward propagation for the model:

>CONV2D -> MAXPOOL -> CONV2D -> MAXPOOL -> FLATTEN -> FULLYCONNECTED -> FULLYCONNECTED -> FULLYCONNECTED

In [0]:
def forward_model(x):    
    mu = 0
    sigma = 0.1
    
    #Layer 1: Convolutional. Input = 32x32x1. Output = 28x28x6.
    W1 = tf.Variable(tf.truncated_normal(shape = [5,5,1,6],mean = mu, stddev = sigma))
    b1 = tf.Variable(tf.zeros(6))
    conv1 = tf.nn.conv2d(x,W1, strides = [1,1,1,1], padding = 'VALID') + b1 
    conv1 = tf.nn.relu(conv1)

    #Pooling. Input = 28x28x6. Output = 14x14x6.
    pool_1 = tf.nn.max_pool(conv1,ksize = [1,2,2,1], strides = [1,2,2,1], padding = 'VALID')
    
    #Layer 2: Convolutional. Output = 10x10x16.
    W2 = tf.Variable(tf.truncated_normal(shape = [5,5,6,16], mean = mu, stddev = sigma))
    b2 = tf.Variable(tf.zeros(16))
    conv2 = tf.nn.conv2d(pool_1, W2, strides = [1,1,1,1], padding = 'VALID') + b2
    conv2 = tf.nn.relu(conv2)

    #Pooling. Input = 10x10x16. Output = 5x5x16.
    pool_2 = tf.nn.max_pool(conv2, ksize = [1,2,2,1], strides = [1,2,2,1], padding = 'VALID') 
    
    #Flatten. Input = 5x5x16. Output = 400.
    fc1 = flatten(pool_2)
    
    #Layer 3: Fully Connected. Input = 400. Output = 120.
    W3 = tf.Variable(tf.truncated_normal(shape = (400,120), mean = mu, stddev = sigma))
    b3 = tf.Variable(tf.zeros(120))
    fc1 = tf.matmul(fc1,W3) + b3
    fc1 = tf.nn.relu(fc1)

    #Layer 4: Fully Connected. Input = 120. Output = 84.
    W4= tf.Variable(tf.truncated_normal(shape = (120,84), mean = mu, stddev = sigma))
    b4 = tf.Variable(tf.zeros(84))
    fc2 = tf.matmul(fc1,W4) + b4
    fc2 = tf.nn.relu(fc2)
    
    #Layer 5: Fully Connected. Input = 84. Output = 10.
    Wo = tf.Variable(tf.truncated_normal(shape = (84,10), mean = mu , stddev = sigma))
    bo = tf.Variable(tf.zeros(10))
    yout = tf.matmul(fc2, Wo) + bo
    return yout

**Placeholder in which we put the data later**

In [0]:
x = tf.placeholder(tf.float32, (None, 32, 32, 1))
y = tf.placeholder(tf.int32, (None))
one_hot_y = tf.one_hot(y, 10)

**Defining Cost, optimizeit & apply backword propagation**

In [0]:
rate = 0.001
EPOCHS = 10
BATCH_SIZE = 100


logits = forward_model(x)
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits,labels=one_hot_y))
optimizer = tf.train.AdamOptimizer(learning_rate = rate)
training_operation = optimizer.minimize(cost)

**Start the Session**
> Apply Batch Gradient desient method with training dataset

In [0]:
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(one_hot_y, 1))
accuracy_operation = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

def evaluate(X_data, y_data):
    num_examples = len(X_data)
    total_accuracy = 0
    sess = tf.get_default_session()
    for i in range(0, num_examples, BATCH_SIZE):
        batch_x, batch_y = X_data[i:i+BATCH_SIZE], y_data[i:i+BATCH_SIZE]
        accuracy = sess.run(accuracy_operation, feed_dict={x: batch_x, y: batch_y})
        total_accuracy += (accuracy * len(batch_x))
    return total_accuracy / num_examples

**Check the Accuracy for the validation dataset**

In [0]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    num_examples = len(X_train)
    
    print("Training...")
    print()
    for i in range(EPOCHS):
        X_train, y_train = shuffle(X_train, y_train)
        for offset in range(0, num_examples, BATCH_SIZE):
            end = offset + BATCH_SIZE
            batch_x, batch_y = X_train[offset:end], y_train[offset:end]
            sess.run(training_operation, feed_dict={x: batch_x, y: batch_y})
            
        validation_accuracy = evaluate(X_validation, y_validation)
        print("Iteration..."+ str(i+1))
        print("Validation Accuracy ="+ str(validation_accuracy*100)+"%")
        print()